# Understanding `itertools.product`

`itertools.product()` (from now on just `product()`) is a powerful tool that makes the solution of assignment 2 from lesson 1 much more concise.

It it's simplest form, `product()` takes two sequences, and generates the cartesian product of its elements.

In [1]:
import itertools as it

list(it.product((1, 2, 3), ('a', 'b')))

[(1, 'a'), (1, 'b'), (2, 'a'), (2, 'b'), (3, 'a'), (3, 'b')]

The number of input sequences, however, is arbitrary.

In [2]:
list(it.product((1, 2), ('a', 'b', 'c'), ('x', 'y')))

[(1, 'a', 'x'),
 (1, 'a', 'y'),
 (1, 'b', 'x'),
 (1, 'b', 'y'),
 (1, 'c', 'x'),
 (1, 'c', 'y'),
 (2, 'a', 'x'),
 (2, 'a', 'y'),
 (2, 'b', 'x'),
 (2, 'b', 'y'),
 (2, 'c', 'x'),
 (2, 'c', 'y')]

The elements don't have to be sequences in the strict sense. Some of them can be "atomic", so to speak.

In [3]:
list(it.product('a', 'b', [1, 2, 3]))

[('a', 'b', 1), ('a', 'b', 2), ('a', 'b', 3)]

In the examples above, we were passing the arguments "one by one". More precisely, we were passing a tuple of arguments. The signature of the function is `itertools.product(*iterables, repeat=1)`, which means that we can pass a sequence of iterables. If we pass an iterable containing such list, we need to unpack it with the `*` operator.

In [4]:
list(it.product(*['a', 'b', [1, 2, 3]]))

[('a', 'b', 1), ('a', 'b', 2), ('a', 'b', 3)]

If we don't use the unpacking operator, the output is not what we would expect.
**TODO** explain this output.

In [5]:
list(it.product(['a', 'b', [1, 2, 3]]))

[('a',), ('b',), ([1, 2, 3],)]

## Assignment 2

Assignment 2 from lesson 1 requires to consider 7-card hands that can contain one or two jokers (a red and a black one). Each joker can be replaced with any card of the same color. The goal of the assignment is to find the best 5-card hand out of all the possible 7-card hands.

My initial solution was complicated and ugly. I would remove the jokers from the deck, and create a list of possible hands in a contrived way. I was mostly bothered by the immutability of tuples, and particularly confused by the case where both jokers are present in the hand.

I found [this solution](https://github.com/lancelote/cs212/blob/master/poker/ps2.py) that is not only much shorter, but also based on `product()`, which I tried, without success, to use in a later discarded solution. Let's consider the 3 hands below. The first one has a black joker and the second one has both, while the third one has none.

In [6]:
hand1 = "6C 7C 8C 9C TC 5C ?B".split()
hand2 = "TD TC 5H 5C 7C ?R ?B".split()
hand3 = "JD TC TH 7C 7D 7S 7H".split()

The solution mentioned above first creates a function that replaces the joker card with all the cards the same color as the joker. If a card is not a joker, the card is returned as a list. The rationale for this approach is not immediately clear. Let's see how it works.

In [7]:
def replace_joker(card):
    "Return a list of possible replacements of a card."
    red_deck = [r+s for r in '23456789TJQKA' for s in 'HD']
    black_deck = [r+s for r in '23456789TJQKA' for s in 'SC']

    if card == "?R":
        return red_deck
    elif card == "?B":
        return black_deck
    else:
        return [card]

print(replace_joker('JD'))
print(replace_joker('?B'))

['JD']
['2S', '2C', '3S', '3C', '4S', '4C', '5S', '5C', '6S', '6C', '7S', '7C', '8S', '8C', '9S', '9C', 'TS', 'TC', 'JS', 'JC', 'QS', 'QC', 'KS', 'KC', 'AS', 'AC']


Why do we need to wrap `card` in a list? Since a card is a string of two caracters, if we don't wrap it into a list, it will be considered a two-element iterable, and we will get the wrong results.

If we apply `replace_joker()` on a hand withoug jokers, we get a list of cards, each one of which is a list in itself.

In [8]:
res3 = [replace_joker(card) for card in hand3]
print(res3)

[['JD'], ['TC'], ['TH'], ['7C'], ['7D'], ['7S'], ['7H']]


If we apply it on a deck with one joker, we get the six non-joker cards, each as a list, and a list with all the black cards.

In [9]:
res1 = [replace_joker(card) for card in hand1]
print(res1)

[['6C'], ['7C'], ['8C'], ['9C'], ['TC'], ['5C'], ['2S', '2C', '3S', '3C', '4S', '4C', '5S', '5C', '6S', '6C', '7S', '7C', '8S', '8C', '9S', '9C', 'TS', 'TC', 'JS', 'JC', 'QS', 'QC', 'KS', 'KC', 'AS', 'AC']]


If we apply `replace_joker()` to a hand with two jokers, we get the 5 non-joker cards, a list with all the red cards, and one with all the black cards.

In [10]:
res2 = [replace_joker(card) for card in hand2]
print(res2)

[['TD'], ['TC'], ['5H'], ['5C'], ['7C'], ['2H', '2D', '3H', '3D', '4H', '4D', '5H', '5D', '6H', '6D', '7H', '7D', '8H', '8D', '9H', '9D', 'TH', 'TD', 'JH', 'JD', 'QH', 'QD', 'KH', 'KD', 'AH', 'AD'], ['2S', '2C', '3S', '3C', '4S', '4C', '5S', '5C', '6S', '6C', '7S', '7C', '8S', '8C', '9S', '9C', 'TS', 'TC', 'JS', 'JC', 'QS', 'QC', 'KS', 'KC', 'AS', 'AC']]


I would have never thought of this solution, and even looking at it, I'm still not sure how to use it. Let's see what happens when we pass these lists to `product()`. In the case without jokers it simply treats each card as its own iterable and returns a tuple with all the cards.

In [11]:
list(it.product(*res3))

[('JD', 'TC', 'TH', '7C', '7D', '7S', '7H')]

In the case with one joker, it returns the cartesian product between the 6 "single" iterators (i.e., the individual cards) and the list of possible replacements.

In [12]:
list(it.product(*res1))

[('6C', '7C', '8C', '9C', 'TC', '5C', '2S'),
 ('6C', '7C', '8C', '9C', 'TC', '5C', '2C'),
 ('6C', '7C', '8C', '9C', 'TC', '5C', '3S'),
 ('6C', '7C', '8C', '9C', 'TC', '5C', '3C'),
 ('6C', '7C', '8C', '9C', 'TC', '5C', '4S'),
 ('6C', '7C', '8C', '9C', 'TC', '5C', '4C'),
 ('6C', '7C', '8C', '9C', 'TC', '5C', '5S'),
 ('6C', '7C', '8C', '9C', 'TC', '5C', '5C'),
 ('6C', '7C', '8C', '9C', 'TC', '5C', '6S'),
 ('6C', '7C', '8C', '9C', 'TC', '5C', '6C'),
 ('6C', '7C', '8C', '9C', 'TC', '5C', '7S'),
 ('6C', '7C', '8C', '9C', 'TC', '5C', '7C'),
 ('6C', '7C', '8C', '9C', 'TC', '5C', '8S'),
 ('6C', '7C', '8C', '9C', 'TC', '5C', '8C'),
 ('6C', '7C', '8C', '9C', 'TC', '5C', '9S'),
 ('6C', '7C', '8C', '9C', 'TC', '5C', '9C'),
 ('6C', '7C', '8C', '9C', 'TC', '5C', 'TS'),
 ('6C', '7C', '8C', '9C', 'TC', '5C', 'TC'),
 ('6C', '7C', '8C', '9C', 'TC', '5C', 'JS'),
 ('6C', '7C', '8C', '9C', 'TC', '5C', 'JC'),
 ('6C', '7C', '8C', '9C', 'TC', '5C', 'QS'),
 ('6C', '7C', '8C', '9C', 'TC', '5C', 'QC'),
 ('6C', '7

In [13]:
list(it.product(*res2))

[('TD', 'TC', '5H', '5C', '7C', '2H', '2S'),
 ('TD', 'TC', '5H', '5C', '7C', '2H', '2C'),
 ('TD', 'TC', '5H', '5C', '7C', '2H', '3S'),
 ('TD', 'TC', '5H', '5C', '7C', '2H', '3C'),
 ('TD', 'TC', '5H', '5C', '7C', '2H', '4S'),
 ('TD', 'TC', '5H', '5C', '7C', '2H', '4C'),
 ('TD', 'TC', '5H', '5C', '7C', '2H', '5S'),
 ('TD', 'TC', '5H', '5C', '7C', '2H', '5C'),
 ('TD', 'TC', '5H', '5C', '7C', '2H', '6S'),
 ('TD', 'TC', '5H', '5C', '7C', '2H', '6C'),
 ('TD', 'TC', '5H', '5C', '7C', '2H', '7S'),
 ('TD', 'TC', '5H', '5C', '7C', '2H', '7C'),
 ('TD', 'TC', '5H', '5C', '7C', '2H', '8S'),
 ('TD', 'TC', '5H', '5C', '7C', '2H', '8C'),
 ('TD', 'TC', '5H', '5C', '7C', '2H', '9S'),
 ('TD', 'TC', '5H', '5C', '7C', '2H', '9C'),
 ('TD', 'TC', '5H', '5C', '7C', '2H', 'TS'),
 ('TD', 'TC', '5H', '5C', '7C', '2H', 'TC'),
 ('TD', 'TC', '5H', '5C', '7C', '2H', 'JS'),
 ('TD', 'TC', '5H', '5C', '7C', '2H', 'JC'),
 ('TD', 'TC', '5H', '5C', '7C', '2H', 'QS'),
 ('TD', 'TC', '5H', '5C', '7C', '2H', 'QC'),
 ('TD', 'T